#  Week 6 text mining

In the assignment of week 6 i am going to use text mining to predict the rating of a dress from online reviews. I need to predict whether dresses reviews are positive(>3 stars) or neutral/negative (<4 stars)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
import numpy as np

## Pre-processing the data

In [2]:
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df.head() #importing the dataframe

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
df = df[(df['Class Name'] == 'Dresses')]
df.dropna(
    axis=0,
    how ='any',
    thresh=None,
    subset = None,
    inplace = True)

df.head()

#Only keep the row with dresses

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses
10,10,1077,53,Dress looks like it's made of cheap material,Dress runs small esp where the zipper area run...,3,0,14,General,Dresses,Dresses


In [4]:
#Divide the positive and negative colomns
def positive_or_negative(x):
    if (x > 3):
         return 'Positive'
        
    else:
          return 'Negative'
        
df['Rate'] = df ['Rating'].apply(positive_or_negative)

In [5]:
#Select the data we need
df=df[["Clothing ID", "Review Text", "Rating", "Class Name", "Rate"]]
df.head()

,Clothing ID,Review Text,Rating,Class Name,Rate
2,1077,I had such high hopes for this dress and reall...,3,Dresses,Negative
5,1080,"I love tracy reese dresses, but this one is no...",2,Dresses,Negative
8,1077,I love this dress. i usually get an xs but it ...,5,Dresses,Positive
9,1077,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,Dresses,Positive
10,1077,Dress runs small esp where the zipper area run...,3,Dresses,Negative


In [6]:
df['Rate'].value_counts()

Positive    4045
Negative    1326
Name: Rate, dtype: int64

## Bag-of-words model and Naïve Bayes

Bag-of-words model
What the Bag-of-Words model does is see the document as a collection of words and count those words. In doing so, it ignores semantics, syntax, morphology and pragmatics. It is a very simple method but effective in many languages.

Naïve Bayes
This model helps us calculate a probability that a text belongs to a certain category. The probability is determined by the frequency of the words

In [7]:
#importing the countvectorizer
from sklearn.feature_extraction.text import CountVectorizer 

#step 1 taking the text from the dataframe
text = df['Review Text'].values.astype('U') 

#Step 2 create the countvecorizer 
vect = CountVectorizer(stop_words='english') 
vect = vect.fit(text) 
vect

#Step 3 the words from the vocabulary

feature_names = vect.get_feature_names() #Get the words from the vocabulary

print(f"There are {len(feature_names)} words in the vocabulary. A selection: {feature_names[500:520]}")

There are 7747 words in the vocabulary. A selection: ['allusion', 'allusione', 'almsot', 'alr', 'alright', 'als', 'altar', 'alter', 'alteration', 'alterations', 'altered', 'altering', 'alternate', 'alternations', 'alternative', 'althetic', 'altho', 'altogether', 'am5', 'amadi']


## Document feature matrix 

In [8]:
#creating a document feature matrix
docu_feat = vect.transform(text)
print(docu_feat[0:500,0:500])

  (1, 7)	1
  (3, 71)	1
  (3, 214)	1
  (12, 480)	1
  (13, 200)	1
  (14, 78)	1
  (14, 408)	1
  (14, 420)	1
  (15, 37)	1
  (16, 3)	1
  (16, 44)	1
  (17, 11)	1
  (17, 236)	1
  (18, 54)	1
  (19, 39)	1
  (21, 59)	1
  (21, 216)	1
  (21, 225)	1
  (22, 229)	1
  (23, 102)	1
  (23, 178)	1
  (23, 222)	1
  (23, 243)	1
  (24, 403)	1
  (26, 11)	2
  :	:
  (475, 96)	1
  (475, 455)	2
  (476, 63)	1
  (476, 200)	1
  (476, 334)	1
  (477, 344)	1
  (478, 156)	1
  (479, 216)	1
  (479, 368)	1
  (482, 309)	2
  (484, 130)	1
  (489, 362)	1
  (490, 112)	1
  (492, 11)	2
  (492, 364)	1
  (493, 50)	1
  (493, 480)	1
  (497, 442)	1
  (498, 334)	1
  (499, 11)	1
  (499, 165)	1
  (499, 187)	1
  (499, 219)	1
  (499, 248)	1
  (499, 451)	1


## Split into a training and a test set

In [9]:
#Creating the model
nb = MultinomialNB()

#Creating the x-matrix
X = docu_feat

#Creating the y-matrix
y= df['Rate']
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)

nb= nb.fit(X_train, y_train)


## Evaluating the model 

In [10]:
#predict the score

y_test_predict = nb.predict(X_test)
nb.score(X_test, y_test)

0.8542183622828784

In [11]:
#make a model of prediction
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_test_predict)
conf_matrix = pd.DataFrame(cm, index = ["Negative (actual)", "Positive (actual)"], columns = ["Negative (predicted)", "Positive (predicted)"])
conf_matrix

,Negative (predicted),Positive (predicted)
Negative (actual),238,168
Positive (actual),67,1139


### Calculate the variables 

What proportion is correctly predict?
$accuracy = \frac{238+1139}{238 + 168 + 67 + 1139} = .85$

How much of the predicted 'positive' is actually positive?
$recall positive = \frac{1139}{1139+ 67} = .94$

How much of the predicted 'negative' is actually negative
$recall negative = \frac{238}{238+ 168} = .59$

In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_predict))

              precision    recall  f1-score   support

    Negative       0.78      0.59      0.67       406
    Positive       0.87      0.94      0.91      1206

    accuracy                           0.85      1612
   macro avg       0.83      0.77      0.79      1612
weighted avg       0.85      0.85      0.85      1612



In [14]:
df ["Rate_Predictions"] = nb.predict(X)
df_contradictions = df[df["Rate"] != df["Rate_Predictions"]]
df_contradictions.head()

,Clothing ID,Review Text,Rating,Class Name,Rate,Rate_Predictions
12,1095,More and more i find myself reliant on the rev...,5,Dresses,Positive,Negative
23,1077,Cute little dress fits tts. it is a little hig...,3,Dresses,Negative,Positive
311,1089,Looks beautiful online but has too much materi...,3,Dresses,Negative,Positive
383,1104,This dress is not what i expected. the bottom ...,3,Dresses,Negative,Positive
417,1083,"I love byron lars dresses, and this design is ...",2,Dresses,Negative,Positive


### To see which reviews are wrong predicted

In [16]:
df_contradictions["Review Text"]

12       More and more i find myself reliant on the rev...
23       Cute little dress fits tts. it is a little hig...
311      Looks beautiful online but has too much materi...
383      This dress is not what i expected. the bottom ...
417      I love byron lars dresses, and this design is ...
                               ...                        
23406    I was very excited to find a fun and lightweig...
23436    I'm 5'6" and it hits me at mid-thigh. the top ...
23475    Cute dress but not for me.  the waist is too h...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
Name: Review Text, Length: 506, dtype: object